In [2]:
import hopsworks

# Explicitly authenticate using the API key
project = hopsworks.login(api_key_value="KpnqasHfb4WsGo1Z.jp7TSl48y51FcnnVvfgVgRk7cot4me3LXYlde0JOWpnzB0clm5x9Fre7tijqumxU")
fs = project.get_feature_store()

print("Connected to Hopsworks Feature Store")


C:\Users\kunal\AppData\Roaming\Python\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2025-01-22 23:20:48,493 INFO: Initializing external client
2025-01-22 23:20:48,496 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-01-22 23:20:52,224 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1211546
Connected to Hopsworks Feature Store


In [4]:
import hopsworks
import pandas as pd
# Load features from the CSV file
features_file = "processed_features_advanced.csv"
features_df = pd.read_csv(features_file)

# Ensure column names match the desired schema
features_df = features_df.rename(columns={
    "datetime": "date",
    "pm2_5": "components_pm2_5",
    "pm10": "components_pm10",
    "temperature": "weather_temperature",
    "humidity": "weather_humidity",
    "wind_speed": "weather_wind_speed",
    "precipitation": "weather_precipitation"
})

# Insert features into the feature group
feature_group = fs.get_or_create_feature_group(
    name="aqi_features",
    version=1,
    description="Features for AQI prediction",
    primary_key=["date"],  # Specify primary key
    online_enabled=True    # Enable online feature storage for real-time inference
)

feature_group.insert(features_df)

print("Features successfully inserted into the feature group!")


Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1211546/fs/1199174/fg/1399146


Uploading Dataframe: 100.00% |██████████| Rows 2113/2113 | Elapsed Time: 00:03 | Remaining Time: 00:00


Launching job: aqi_features_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1211546/jobs/named/aqi_features_1_offline_fg_materialization/executions
Features successfully inserted into the feature group!


In [18]:
import hopsworks

# Explicitly authenticate using the API key
project = hopsworks.login(api_key_value="KpnqasHfb4WsGo1Z.jp7TSl48y51FcnnVvfgVgRk7cot4me3LXYlde0JOWpnzB0clm5x9Fre7tijqumxU")
fs = project.get_feature_store()

print("Connected to Hopsworks Feature Store")

# Get the feature group
feature_group = fs.get_feature_group(name="aqi_features", version=1)

# Fetch data as a Pandas DataFrame
data = feature_group.read()

# Separate features and target
features = data.drop(columns=["aqi"])  # Replace 'aqi' with your actual target column
target = data["aqi"]

print("Data fetched successfully from the feature store!")


2025-01-22 23:46:18,364 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-01-22 23:46:18,396 INFO: Initializing external client
2025-01-22 23:46:18,399 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-01-22 23:46:21,707 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1211546
Connected to Hopsworks Feature Store
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (2.44s) 
Data fetched successfully from the feature store!


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Step 1: Transform 'date' column into numeric features
features["date"] = pd.to_datetime(features["date"], errors="coerce")

# Extract useful numeric features from 'date'
features["date_year"] = features["date"].dt.year
features["date_month"] = features["date"].dt.month
features["date_day"] = features["date"].dt.day
features["date_hour"] = features["date"].dt.hour
features["date_minute"] = features["date"].dt.minute

# Convert 'date' to a numeric timestamp
features["date_timestamp"] = features["date"].view('int64') // 10**9

# Drop the original 'date' column
features = features.drop(columns=["date"], errors="ignore")  # Added errors="ignore" for safety

# Step 3: Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)





KeyError: "'aqi' column not found in the DataFrame."

In [20]:
#Random Forest Regressor

# Step 4: Train the Model
model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)

# Step 5: Evaluate the Model
y_pred = model.predict(X_test)
rmse = mean_squared_error(y_test, y_pred, squared=False)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Model Performance:\nRMSE: {rmse}\nMAE: {mae}\nR²: {r2}")

Model Performance:
RMSE: 0.039411907861542535
MAE: 0.006178570340459369
R²: 0.9983509512167169


In [30]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.svm import SVR
import numpy as np
import joblib
import hopsworks

# Define models to train
models = {
    "Random_Forest": RandomForestRegressor(random_state=42),
    "Gradient_Boosting": GradientBoostingRegressor(random_state=42),
    "Linear_Regression": LinearRegression(),
    "Ridge_Regression": Ridge(alpha=1.0),
    "Support_Vector_Regressor": SVR(kernel="rbf")
}

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Step 4: Train and Evaluate Each Model
results = []
best_model = None
best_model_name = None
best_rmse = float("inf")  # Initialize with infinity for comparison

for model_name, model in models.items():
    # Train the model
    model.fit(X_train, y_train)
    
    # Make predictions
    y_pred = model.predict(X_test)
    
    # Evaluate the model
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    # Store results
    results.append({
        "Model": model_name,
        "RMSE": rmse,
        "MAE": mae,
        "R²": r2
    })

    # Check for the best model
    if rmse < best_rmse:
        best_rmse = rmse
        best_model = model
        best_model_name = model_name

    print(f"Model: {model_name}")
    print(f"  RMSE: {rmse}")
    print(f"  MAE: {mae}")
    print(f"  R²: {r2}\n")

# Step 5: Store the Best Model in Hopsworks Model Registry
if best_model:
    print(f"The best model is: {best_model_name} with RMSE: {best_rmse}")

    # Save the best model locally
    joblib.dump(best_model, "best_model.pkl")

    # Connect to the Hopsworks Model Registry
    project = hopsworks.login(api_key_value="KpnqasHfb4WsGo1Z.jp7TSl48y51FcnnVvfgVgRk7cot4me3LXYlde0JOWpnzB0clm5x9Fre7tijqumxU")  # Use your API key
    mr = project.get_model_registry()

    # Register the model metadata
    model_registry = mr.python.create_model(
        name=best_model_name,  # Model name complies with naming conventions
        metrics={"rmse": best_rmse, "mae": mae, "r2": r2},
        description=f"Best model for AQI prediction: {best_model_name}"
    )

    # Upload the saved model file to the registry
    model_registry.save("best_model.pkl")
    print(f"Model registered successfully in Hopsworks Model Registry: {best_model_name}")

# Step 6: Display Results
results_df = pd.DataFrame(results)
print("Summary of Model Performance:")
print(results_df)


Model: Random_Forest
  RMSE: 0.039411907861542535
  MAE: 0.006178570340459369
  R²: 0.9983509512167169

Model: Gradient_Boosting
  RMSE: 0.013199563117497901
  MAE: 0.0023168300817206495
  R²: 0.9998150315318372

Model: Linear_Regression
  RMSE: 9.674801162219957e-12
  MAE: 7.249560815068621e-12
  R²: 1.0

Model: Ridge_Regression
  RMSE: 0.008999796041045022
  MAE: 0.005969509625451406
  R²: 0.9999140107976446



Model: Support_Vector_Regressor
  RMSE: 1.1216669627973712
  MAE: 0.6909205057930925
  R²: -0.33569236288031834

The best model is: Linear_Regression with RMSE: 9.674801162219957e-12
2025-01-23 01:28:55,335 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-01-23 01:28:55,345 INFO: Initializing external client
2025-01-23 01:28:55,347 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-01-23 01:28:58,627 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1211546


Uploading: 100.000%|██████████| 2968/2968 elapsed<00:02 remaining<00:00:05,  1.12s/it]
Model export complete: 100%|██████████| 6/6 [00:10<00:00,  1.70s/it]                   

Model created, explore it at https://c.app.hopsworks.ai:443/p/1211546/models/Linear_Regression/1
Model registered successfully in Hopsworks Model Registry: Linear_Regression
Summary of Model Performance:
                      Model          RMSE           MAE        R²
0             Random_Forest  3.941191e-02  6.178570e-03  0.998351
1         Gradient_Boosting  1.319956e-02  2.316830e-03  0.999815
2         Linear_Regression  9.674801e-12  7.249561e-12  1.000000
3          Ridge_Regression  8.999796e-03  5.969510e-03  0.999914
4  Support_Vector_Regressor  1.121667e+00  6.909205e-01 -0.335692
